# NmapBuddy - Automated Port & Vulnerability Scanner


## 📚 Table of Contents
1. [Overview](#overview)
2. [Setup](#setup)
3. [Timestamp + Logging](#timestamp)
4. [Target Selection](#target)
5. [Nmap Scan Execution](#scan)
6. [Output & Results](#results)
7. [Future Features](#future)


# NmapBuddy - Automated Port and Vulnerability Scanner
---
**Created by:** Tamieka Horne  
**Date Started:** 4/27/2025  

This project automates port scanning and basic vulnerability detection using Nmap and Python.  
It is designed to streamline manual security assessments by allowing users to quickly identify open ports and potential vulnerabilities on a target system or network.


## 🛠️ Tools & Technologies

This project was built using:
- **Python 3.x**
- **Nmap** – for performing port and vulnerability scans
- **subprocess module** – to execute shell commands from within Python
- **datetime module** – for timestamping logs
- **Jupyter Notebook** – for interactive scripting, documentation, and analysis

These tools were chosen for their accessibility, effectiveness, and ability to support real-time iteration and automation in security assessments.


In [1]:
# Install the python-nmap package
%pip install python-nmap

import nmap

# Initialize the scanner
scanner = nmap.PortScanner()

# Define target and ports
target = 'scanme.nmap.org'
ports = '20-100'

# Run a basic scan
print(f"Scanning {target} for ports {ports}...")

scanner.scan(target, ports)

# Display the scan results
for host in scanner.all_hosts():
    print(f"\nHost: {host} ({scanner[host].hostname()})")
    print(f"State: {scanner[host].state()}")

    for protocol in scanner[host].all_protocols():
        print(f"\nProtocol: {protocol}")
        ports = scanner[host][protocol].keys()
        for port in ports:
            print(f"Port: {port}\tState: {scanner[host][protocol][port]['state']}")


  Preparing metadata (setup.py) ... done
  Created wheel for python-nmap: filename=python_nmap-0.7.1-py2.py3-none-any.whl size=20633 sha256=1e08ee243e0da8594193da955d8c4d1f9dacd39645ea8f9fe009505fe80a0fcd
  Stored in directory: /Users/tamiekahorne/Library/Caches/pip/wheels/06/fc/d4/0957e1d9942e696188208772ea0abf909fe6eb3d9dff6e5a9e
Successfully built python-nmap
Note: you may need to restart the kernel to use updated packages.
Scanning scanme.nmap.org for ports 20-100...

Host: 45.33.32.156 (scanme.nmap.org)
State: up

Protocol: tcp
Port: 22	State: open
Port: 25	State: filtered
Port: 53	State: open
Port: 80	State: open


## Basic Nmap Scan
This cell runs a simple Nmap port scan on `scanme.nmap.org` from ports 20 to 100.
It prints the open ports and the status of the target host.
